# VacationPy
----

#### Note to Grader
* Please run this code in Jupyter notebooks to view the gmap data. Thank you.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
newdf = pd.read_csv("../WeatherPy/outputs/weather.csv")
newdf

,id,city,country,lat,long,temp,humidity,cloudiness,windspeed
0,2155415,New Norfolk,AU,-42.7826,147.0587,53.20,73,45,1.99
1,2400547,Gamba,GA,-2.6500,10.0000,82.04,75,98,11.27
2,1501377,Kungurtug,RU,50.5994,97.5228,2.21,89,52,1.72
3,1794794,Shouguang,CN,36.8800,118.7375,37.29,56,0,7.94
4,2656258,Barrhead,GB,55.7992,-4.3929,37.96,74,93,5.99
...,...,...,...,...,...,...,...,...,...
495,1235846,Matara,LK,5.9485,80.5353,77.74,85,24,3.78
496,5579368,Loveland,US,40.3978,-105.0750,49.95,42,1,1.01
497,3397893,Itacoatiara,BR,-3.1431,-58.4442,82.69,78,99,2.33
498,6089245,Norman Wells,CA,65.2820,-126.8329,-14.76,64,90,19.57


In [4]:
# Store latitude and longitude in locations
locations = newdf[["lat", "long"]]

# Fill NaN values and convert to float
humidity = newdf["humidity"].astype(float)
humidity

0      73.0
1      75.0
2      89.0
3      56.0
4      74.0
       ... 
495    85.0
496    42.0
497    78.0
498    64.0
499    31.0
Name: humidity, Length: 500, dtype: float64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
IdealVacDf = newdf.loc[(newdf['temp'] < 80) & # change to 80
                       (newdf['temp'] > 70) & # change to 70 
                       (newdf['cloudiness'] < 5) & # change to 5 
                       (newdf['windspeed'] < 10), # change to 10
                        ['city', 'country','lat', 'long', 'temp', 'humidity', 'cloudiness', 'windspeed']]
IdealVacDf

,city,country,lat,long,temp,humidity,cloudiness,windspeed
18,Qandala,SO,11.4720,49.8728,77.49,60,0,6.87
32,Najrān,SA,17.4924,44.1277,72.25,16,0,8.39
43,Sharjah city,AE,25.3573,55.4033,79.68,47,0,4.61
239,Cabo San Lucas,MX,22.8909,-109.9124,76.26,66,1,4.61
274,Chabahar,IR,25.2919,60.6430,75.96,66,1,9.19
322,Chauk,MM,20.8833,94.8167,73.35,64,4,1.97
367,Aldama,MX,22.9167,-98.0667,71.28,72,0,3.51
423,Morón,AR,-34.6534,-58.6198,75.06,57,0,5.01
444,Bandar-e Lengeh,IR,26.5579,54.8807,75.31,60,0,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = IdealVacDf
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df

,city,country,lat,long,temp,humidity,cloudiness,windspeed,Hotel Name
18,Qandala,SO,11.4720,49.8728,77.49,60,0,6.87,
32,Najrān,SA,17.4924,44.1277,72.25,16,0,8.39,
43,Sharjah city,AE,25.3573,55.4033,79.68,47,0,4.61,
239,Cabo San Lucas,MX,22.8909,-109.9124,76.26,66,1,4.61,
274,Chabahar,IR,25.2919,60.6430,75.96,66,1,9.19,
322,Chauk,MM,20.8833,94.8167,73.35,64,4,1.97,
367,Aldama,MX,22.9167,-98.0667,71.28,72,0,3.51,
423,Morón,AR,-34.6534,-58.6198,75.06,57,0,5.01,
444,Bandar-e Lengeh,IR,26.5579,54.8807,75.31,60,0,4.61,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "radius": 5000, #change to 5000
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    latsearch = row['lat']
    lonsearch = row['long']
    
    # add keyword to params dict
    params["location"] = str(latsearch)+","+str(lonsearch)
    
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
    try:
        print(f"First lodging found is called {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Missing field/result... skipping.
------------
First lodging found is called محمد هادي ال فطيح.
------------
First lodging found is called Swiss Belhotel Sharjah.
------------
First lodging found is called Hotel Tesoro Los Cabos.
------------
First lodging found is called هتل گدروشیا.
------------
First lodging found is called Bo Khin Guest House.
------------
First lodging found is called CASA CANYE.
------------
First lodging found is called Hotel Morón.
------------
First lodging found is called کافی نت پارس.
------------


In [9]:
hotel_df

,city,country,lat,long,temp,humidity,cloudiness,windspeed,Hotel Name
18,Qandala,SO,11.4720,49.8728,77.49,60,0,6.87,
32,Najrān,SA,17.4924,44.1277,72.25,16,0,8.39,محمد هادي ال فطيح
43,Sharjah city,AE,25.3573,55.4033,79.68,47,0,4.61,Swiss Belhotel Sharjah
239,Cabo San Lucas,MX,22.8909,-109.9124,76.26,66,1,4.61,Hotel Tesoro Los Cabos
274,Chabahar,IR,25.2919,60.6430,75.96,66,1,9.19,هتل گدروشیا
322,Chauk,MM,20.8833,94.8167,73.35,64,4,1.97,Bo Khin Guest House
367,Aldama,MX,22.9167,-98.0667,71.28,72,0,3.51,CASA CANYE
423,Morón,AR,-34.6534,-58.6198,75.06,57,0,5.01,Hotel Morón
444,Bandar-e Lengeh,IR,26.5579,54.8807,75.31,60,0,4.61,کافی نت پارس


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,#,
    info_box_content=hotel_info
    #info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure(center=(0, 0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))